In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import documents as docs
import textcomparisons as tc
import textcomparisons_threads as machine
import random
import time

start = time.time()

data_folder = "data"
results_folder = "results"
para_sep = "###"

ac = tc.ArticleComparisons(thresh_jaccard = .5, thresh_same_sent = .9, thresh_same_doc = .25)

article_df = pd.read_pickle(os.path.join(data_folder, "article_df_20180715"))

display(article_df.head(2))
print("Setup time: %d seconds" % np.round(time.time() - start))

,id,publisher,scrape_time,on_homepage,url,title,text,fingerprint,event,doc,paywall
0,0,wsbtv.com,2018-07-16T01:02:26-04:00,True,https://www.wsbtv.com/news/local/cobb-county/p...,"Police: Missing teen with autism found, reunit...","Police: Missing teen with autism found, reunit...","{'wordCounts': {'1533': 10, '367': 3, '2619': ...",NaN,None,0.0
1,1,wn.com,2018-07-16T04:51:37-04:00,False,https://wn.com/Apostolic_Constitutions,Apostolic s,Apostolic Constitutions ### back to playlist #...,"{'wordCounts': {'147633': 5, '5029': 6, '546':...",NaN,None,0.0


Setup time: 2 seconds


In [ ]:
n = 500
start = time.time()
#sample = random.sample(list(article_df["id"]), n)
sample = [i for i in range(n)]#f article_df.loc[i, "event"] == 1]
elapsed = round(time.time() - start, 2)
# 6.33
article_dict_spacy = ac.dict_by_ids(article_df, sample, para_sep = para_sep, parser = "spacy")
print("Time taken to run cell: %f s (%f s per document)" % (elapsed, elapsed/n))
print(len(sample), sample)

In [ ]:
comparer = machine.ParallelComparisons()
mat = comparer.run(article_dict_spacy)

In [5]:
mat

array([[1.        , 0.        , 0.01328904, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01328904, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])